## Configuração (Somente quando necessário)

In [ ]:
! pip install pymysql;

## Restaurar danco de dados

In [1]:
import pymysql
import os
import time

def listar_databases_atuais(cursor):
    cursor.execute('SHOW DATABASES')
    bancos_sommusgestor = [db[0] for db in cursor.fetchall() if 'sommusgestor' in db[0]]
    return bancos_sommusgestor

def deletar_database(cursor, nome):
    cursor.execute(f"DROP DATABASE {nome}")

def criar_database(cursor, nome):
    if(nome.startswith("sommusgestor") and not nome.endswith("_old")):
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nome}")

def restaurar(banco, host, porta, usuario, password, diretorio):
    if(banco.startswith("sommusgestor") and not banco.endswith("_old")):
        comando = f'mysql -h {host} -P {porta} -u {usuario} -p{password} {banco} < {diretorio}/{banco}.sql'
        os.popen(comando)

def tabela_existe(cursor):
    cursor.execute(f"SHOW TABLES LIKE 'versao'")
    resultado = cursor.fetchall()
    return bool(resultado)

def analisar_final_restauracao(cursor, banco):
    if(banco.startswith("sommusgestor") and banco != "sommusgestor_hangfire" and not banco.endswith("_old")):
        cursor.execute(f"USE {banco}")
        while not tabela_existe(cursor):
            time.sleep(5)

In [2]:
HOST = 'localhost'
PORTA = 3307
USUARIO = 'root'
PASSWORD = 'root'
PASTA = 'backups'
SUBPASTA = 'bosch_car_2023-12-11_14-40-19'
DIRETORIO_BACKUP = f'{PASTA}/{SUBPASTA}'

if __name__ == '__main__':
    try:
        with pymysql.connect(host=HOST, user=USUARIO, password=PASSWORD, port=PORTA).cursor() as cursor:
            bancos_sommusgestor = listar_databases_atuais(cursor)
            
            for banco in bancos_sommusgestor:
                deletar_database(cursor, banco)

            arquivos_sql_backup = os.listdir(DIRETORIO_BACKUP)

            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                criar_database(cursor, nome_arquivo_sem_extensao)

            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                restaurar(nome_arquivo_sem_extensao, HOST, PORTA, USUARIO, PASSWORD, DIRETORIO_BACKUP)

            for arquivo in arquivos_sql_backup:
                nome_arquivo_sem_extensao = os.path.splitext(arquivo)[0]
                analisar_final_restauracao(cursor, nome_arquivo_sem_extensao)

        print("✅ Restauração finalizada!")

    except Exception as e:
        print(f'⛔ Exceção lançada: {e}')

 

✅ Restauração finalizada!
